## Install dependencies

In [0]:
!pip install -q -U datasets
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets

## 1. Loading dataset
Let's load a meaning representation dataset, and fine-tune Mistral on that. This is a great fine-tuning dataset as it teaches the model a unique form of desired output on which the base model performs poorly out-of-the box, so it's helpful to easily and inexpensively gauge whether the fine-tuned model has learned well. (Sources: here and here) (In contrast, if you fine-tune on a fact-based dataset, the model may already do quite well on that, and gauging learning is less obvious / may be more computationally expensive.)

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset('Unbabel/TowerBlocks-v0.2', split='train')
eval_dataset = load_dataset('Unbabel/TowerBlocks-v0.2', split='validation')
test_dataset = load_dataset('Unbabel/TowerBlocks-v0.2', split='test')